In [65]:
import pandas as pd
import json
import ast
from itertools import chain
from datetime import datetime
from itertools import combinations
import pandas as pd
from datetime import datetime
import hashlib

In [5]:
hadoop_result = pd.read_csv("./hadoop_result_matches.csv")
hadoop_result.head(5)

,window_time,selection_strategy,engine,handle_time,matches,memory_usage,record
0,10,STAM,pacep,0.677659,0,154889.250000,[]
1,10,STAM,pacepi,0.657474,0,155140.406250,[]
2,10,STAM,pacepm,0.695945,0,155264.125000,[]
3,20,STAM,pacep,0.756918,4,155573.703125,"[{'a': [['2025-01-26 23:43:57', 741]], 'b': [[..."
4,20,STAM,pacepi,0.768228,4,155959.750000,"[{'a': [['2025-01-26 23:43:57', 741]], 'b': [[..."


In [59]:
percision = {}
for idex, row in hadoop_result.iterrows():
    TP = 0
    FP = 0
    record = row["record"]
    r = ast.literal_eval(record)
    for result in r:
        a = result['a']
        b = result['b']
        c = result['c']
        # 验证时间顺序是否正确
        time_ok = True
        ans_time = datetime.min
        for ans in chain(a,b,c):
            now_time = datetime.strptime(ans[0], "%Y-%m-%d %H:%M:%S")
            if  now_time > ans_time:
                ans_time = now_time
            else:
                FP += 1
                time_ok = false
        
        # 验证是否为递减序列
        if time_ok:
            b_value = 0
            for b_item in b:
                if b_item[1] >= b_value:
                    b_value = b_item[1]
                else:
                    FP += 1
                    time_ok = False
                    break
        
        if time_ok:
            TP += 1
        
    
    window_time = row["window_time"]
    engine = row["engine"]
    if engine not in percision:
        percision[engine] = {}
        
    if window_time not in percision[engine]:
        percision[engine][window_time] = {}
    percision[engine][window_time] = {"TP":TP, "FP":FP, "percision": (TP/(TP+FP)) if (TP + FP) != 0 else 1}
percision

{'pacep': {10: {'TP': 0, 'FP': 0, 'percision': 1},
  20: {'TP': 4, 'FP': 0, 'percision': 1.0},
  30: {'TP': 32, 'FP': 0, 'percision': 1.0},
  40: {'TP': 79, 'FP': 0, 'percision': 1.0},
  50: {'TP': 157, 'FP': 0, 'percision': 1.0},
  60: {'TP': 321, 'FP': 0, 'percision': 1.0},
  70: {'TP': 587, 'FP': 0, 'percision': 1.0},
  80: {'TP': 926, 'FP': 0, 'percision': 1.0},
  90: {'TP': 1232, 'FP': 0, 'percision': 1.0},
  100: {'TP': 1713, 'FP': 0, 'percision': 1.0},
  150: {'TP': 8846, 'FP': 0, 'percision': 1.0},
  200: {'TP': 20623, 'FP': 0, 'percision': 1.0}},
 'pacepi': {10: {'TP': 0, 'FP': 0, 'percision': 1},
  20: {'TP': 4, 'FP': 0, 'percision': 1.0},
  30: {'TP': 32, 'FP': 0, 'percision': 1.0},
  40: {'TP': 79, 'FP': 0, 'percision': 1.0},
  50: {'TP': 157, 'FP': 0, 'percision': 1.0},
  60: {'TP': 321, 'FP': 0, 'percision': 1.0},
  70: {'TP': 587, 'FP': 0, 'percision': 1.0},
  80: {'TP': 926, 'FP': 0, 'percision': 1.0},
  90: {'TP': 1232, 'FP': 0, 'percision': 1.0},
  100: {'TP': 1713, '

In [60]:
hadoop = pd.read_csv('./hadoop.csv')
hadoop.head(5)

,window_time,selection_strategy,engine,handle_time,matches,memory_usage
0,20,STAM,pacep,0.725869,4,128603.125000
1,20,STAM,pacepi,0.705484,4,129351.593750
2,20,STAM,pacepm,0.741644,4,129851.968750
3,30,STAM,pacep,0.738778,32,129574.875000
4,30,STAM,pacepi,0.772829,32,130410.015625


In [70]:
# 通过字典获取 TP 和 FP 值的函数
def get_tp_fp(row):
    strategy = row['selection_strategy']
    window_time = row['window_time']
    if engine in percision and window_time in percision[engine]:
        tp_fp = percision[engine][window_time]
        return pd.Series([tp_fp['TP'], tp_fp['FP']], index=['TP', 'FP'])
    else:
        return pd.Series([0, 0], index=['TP', 'FP'])

# 添加 TP 和 FP 列到 DataFrame
hadoop[['TP', 'FP']] = hadoop.apply(get_tp_fp, axis=1)
hadoop.head(5)

,window_time,selection_strategy,engine,handle_time,matches,memory_usage,TP,FP
0,20,STAM,pacep,0.725869,4,128603.125000,4,0
1,20,STAM,pacepi,0.705484,4,129351.593750,4,0
2,20,STAM,pacepm,0.741644,4,129851.968750,4,0
3,30,STAM,pacep,0.738778,32,129574.875000,32,0
4,30,STAM,pacepi,0.772829,32,130410.015625,32,0


In [64]:
hadoop_log = pd.read_csv('./hadoop_log.csv')
hadoop_log.head(5)

,type,value,timestamp
0,map_end,772,2025-01-26 23:28:54
1,shuffle_end,22,2025-01-26 23:28:57
2,reducer_start,914,2025-01-26 23:29:00
3,reducer_end,314,2025-01-26 23:29:04
4,load_std,1,2025-01-26 23:29:08


In [85]:
def find_valid_combinations(df, time_limit):
    """
    找到满足 A, C 约束的所有 B 组合，并存入哈希集合
    :param df: Pandas DataFrame，包含 type, value, timestamp
    :param time_limit: 时间限制，C.time - A.time 必须小于该值
    :return: 满足条件的哈希集合
    """
    
    # 将 timestamp 转换为 datetime 格式
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    # 结果集合（存储哈希值）
    valid_combinations_set = set()
    
    # 找到所有 A（reducer_start）和 C（reducer_end）对
    A_events = df[df['type'] == 'reducer_start']
    C_events = df[df['type'] == 'reducer_end']
    
    for _, A in A_events.iterrows():
        for _, C in C_events.iterrows():
            if A['timestamp'] < C['timestamp'] and (C['timestamp'] - A['timestamp']).total_seconds() <= time_limit:
                # 找到 A.timestamp 和 C.timestamp 之间的所有 B（load_std）
                B_candidates = df[
                    (df['type'] == 'load_std') & 
                    (df['timestamp'] >= A['timestamp']) & 
                    (df['timestamp'] <= C['timestamp'])
                ]
                
                # 生成所有长度 > 2 的 B 组合
                B_candidates = B_candidates.sort_values(by='timestamp')  # 确保时间顺序
                B_list = B_candidates.to_dict('records')  # 转换为字典列表
                
                # 遍历所有可能的组合
                for r in range(2, len(B_list) + 1):
                    for B_comb in combinations(B_list, r):
                        # 确保时间顺序递增
                        timestamps = [b['timestamp'] for b in B_comb]
                        if all(timestamps[i] >= timestamps[i - 1] for i in range(1, len(timestamps))):
                            value = [b['value'] for b in B_comb]
                            if all(value[i] >= value[i - 1] for i in range(1, len(value))):
                                # 生成组合并哈希化
                                combination = (A.to_dict(), list(B_comb), C.to_dict())
                                hash_value = hashlib.sha256(str(combination).encode()).hexdigest()
                                valid_combinations_set.add(hash_value)
    
    return valid_combinations_set

result_set = find_valid_combinations(hadoop_log, 100)

# 输出结果
print("满足条件的哈希集合：")
print(len(result_set))

满足条件的哈希集合：
1713
